#### This notebook attempts to train a DenseNet121 pre-trained model.

In [18]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.preprocessing import image
from joblib import dump
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from sklearn.utils.class_weight import compute_class_weight

    Define Constants

In [19]:
BATCH_SIZE = 32
IMG_WIDTH = 224
IMG_HEIGHT = 224
EPOCHS = 40

    Data Directories

In [20]:
train_dir = '../datasets/train'
test_dir = '../datasets/test'

    Perform Data Augmentation for the Data

In [21]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

    Preprocess the test data in simple manner.

In [22]:
test_datagen = ImageDataGenerator(rescale=1./255)

    Get training images in batches using train_data generator.

In [23]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'  
)

Found 4173 images belonging to 2 classes.


    Get validation images in batches using validation generator.

In [24]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

Found 1043 images belonging to 2 classes.


    One way to fight imbalance is to have class weights defined and add penalty for misclassfying minority class.
    - Generate samples, find weights and then use that to fit the model.

In [25]:
temp_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',  
    shuffle=False  
)


Found 5216 images belonging to 2 classes.


    Compute the class weights

In [26]:
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(temp_generator.classes),
    y=temp_generator.classes
)

    Convert weights into dictionary

In [27]:
class_weight_dict = {i : class_weights[i] for i in range(len(class_weights))}

    Load the DenseNet121 model, pretrained on ImageNet

In [28]:
# Load the DenseNet121 base model, pretrained on ImageNet
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False


    The model performs terribly bad, due to overfitting.
        - We must prevent it from overfitting. By using early stopping.

In [29]:
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor the validation loss
    patience=5,          # Number of epochs with no improvement after which training will be stopped
    verbose=1,           # To print messages when the callback takes an action
    restore_best_weights=True  # Optional: Restore model weights from the epoch with the best value of the monitored quantity
)

    Add new layers on top of DenseNet

In [30]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x) 

    Now train this model:

In [31]:
model = Model(inputs=base_model.input, outputs=predictions)

    Compile this model

In [32]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    Train this modified ResNet50

In [33]:
history = model.fit(train_generator, 
                    epochs=EPOCHS, 
                    validation_data=validation_generator, 
                    class_weight=class_weight_dict,
                    callbacks=[early_stopping])


Epoch 1/40
131/131 [==============================] - 201s 2s/step - loss: 0.2872 - accuracy: 0.8730 - val_loss: 0.6381 - val_accuracy: 0.7593
Epoch 2/40
131/131 [==============================] - 187s 1s/step - loss: 0.1947 - accuracy: 0.9243 - val_loss: 0.1991 - val_accuracy: 0.9214
Epoch 3/40
131/131 [==============================] - 183s 1s/step - loss: 0.1673 - accuracy: 0.9300 - val_loss: 0.1825 - val_accuracy: 0.9223
Epoch 4/40
131/131 [==============================] - 183s 1s/step - loss: 0.1955 - accuracy: 0.9195 - val_loss: 0.1600 - val_accuracy: 0.9367
Epoch 5/40
131/131 [==============================] - 183s 1s/step - loss: 0.1744 - accuracy: 0.9276 - val_loss: 0.1553 - val_accuracy: 0.9415
Epoch 6/40
131/131 [==============================] - 186s 1s/step - loss: 0.1439 - accuracy: 0.9401 - val_loss: 0.3708 - val_accuracy: 0.8639
Epoch 7/40
131/131 [==============================] - 188s 1s/step - loss: 0.1567 - accuracy: 0.9324 - val_loss: 0.2537 - val_accuracy: 0.8878